In [1]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import DeepLake

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
os.environ["ACTIVELOOP_TOKEN"] = getpass.getpass("Activeloop Token:")

OpenAI API Key:········
Activeloop Token:········


In [3]:
llm = OpenAI(temperature=0)

In [4]:
embeddings = OpenAIEmbeddings(disallowed_special=())


Only uncomment if you want to load research papers as PDFs

In [46]:
# import os
# import nltk
# import PyPDF2

# # Directory path where the research papers are stored
# directory = 'papers/'

# # Function to extract text from a PDF file
# def extract_text_from_pdf(file_path):
#     with open(file_path, 'rb') as file:
#         reader = PyPDF2.PdfReader(file)
#         text = ''
#         for page_num in range(len(reader.pages)):
#             page = reader.pages[page_num]
#             text += page.extract_text()
#         return text

# # Function to count tokens in text
# def count_tokens(text):
#     tokens = nltk.word_tokenize(text)
#     return len(tokens)

# # Download the necessary resource for tokenization
# nltk.download('punkt')

# # Iterate over files in the directory
# file_list = os.listdir(directory)
# total_tokens = 0

# for file_name in file_list:
#     # Filter files with specific extensions if needed
#     if file_name.endswith('.pdf'):
#         file_path = os.path.join(directory, file_name)
#         try:
#             text = extract_text_from_pdf(file_path)
#             num_tokens = count_tokens(text)
#             print(f"File: {file_name}\tTokens: {num_tokens}")
#             total_tokens += num_tokens
#         except Exception as e:
#             print(f"Error reading file: {file_path}\n{str(e)}")

# print(f"Total Tokens: {total_tokens}")


In [ ]:
# loader = PyPDFDirectoryLoader("papers")
# docs = loader.load()

In [ ]:
# text_splitter = CharacterTextSplitter(chunk_size=10000, chunk_overlap=500)
# texts = text_splitter.split_documents(docs)

Loading txt file with the relevant parts from the research paper

In [16]:
loader = TextLoader("Research papers_Clean.docx.txt")
docs = loader.load()

In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=25)
texts = text_splitter.split_documents(docs)

In [23]:
# # import os
# import nltk

# # Directory path where the research papers are stored
# file_path = 'Research papers_Clean.docx.txt'

# # Function to extract text from a PDF file

# def extract_text_from_txt(file_path):
#     with open(file_path, 'r') as file:
#         text = file.readlines()
#     text= listToString(text)

# # Function to count tokens in text
# def count_tokens(text):
#     tokens = nltk.word_tokenize(text)
#     return len(tokens)

# # Download the necessary resource for tokenization
# nltk.download('punkt')

# try:
#     text = extract_text_from_txt(file_path)
#     num_tokens = count_tokens(text)
#     print(f"File: {file_name}\tTokens: {num_tokens}")
#     total_tokens += num_tokens
# except Exception as e:
#     print(f"Error reading file: {file_path}\n{str(e)}")

# print(f"Total Tokens: {total_tokens}")

In [24]:
# num_docs = len(texts)

# num_tokens_first_doc = llm.get_num_tokens(texts[0].page_content)

# print (f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens")

Only uncomment if you want to create a new dataset

In [45]:
# username = "commutatus"  # replace with your username from app.activeloop.ai
# db = DeepLake(
#     dataset_path=f"hub://{username}/research_papers_summary_doc",
#     embedding_function=embeddings,
# )
# db.add_documents(texts)

In [26]:
db = DeepLake(
    dataset_path="hub://commutatus/research_papers_summary_doc",
    read_only=True,
    embedding_function=embeddings,
)

-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/commutatus/research_papers_summary_doc



|

hub://commutatus/research_papers_summary_doc loaded successfully.

Deep Lake Dataset in hub://commutatus/research_papers_summary_doc already exists, loading from the storage
Dataset(path='hub://commutatus/research_papers_summary_doc', read_only=True, tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape      dtype  compression
  -------   -------   -------    -------  ------- 
 embedding  generic  (94, 1536)  float32   None   
    ids      text     (94, 1)      str     None   
 metadata    json     (94, 1)      str     None   
   text      text     (94, 1)      str     None   


In [27]:
from nltk.tokenize import sent_tokenize
def chunk_text_by_sentence(file_path, chunk_size):
    with open(file_path, 'r') as file:
        text = file.readlines()

    text= listToString(text)
    chunks = []

    sentences = sent_tokenize(text)
    num_sentences = len(sentences)

    for i in range(0, num_sentences, chunk_size):
        chunk = '  '.join(sentences[i:i+chunk_size])
        chunks.append(chunk)
    return chunks


In [28]:
retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["mmr"] = True
retriever.search_kwargs["k"] = 10

In [29]:
def listToString(s):
 
    # initialize an empty string
    str1 = ""
 
    # traverse in the string
    for ele in s:
        str1 += ele
 
    # return string
    return str1

In [30]:
import PyPDF2
from nltk.tokenize import sent_tokenize
import nltk
def chunk_pdf_by_sentence(pdf_path, chunk_size):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        
        chunks = []
        for page_number in range(num_pages):
            page = reader.pages[page_number]
            text = page.extract_text()
            print("TEXT: " + text)
            print("*******")
            
            sentences = sent_tokenize(text)
            num_sentences = len(sentences)
            
            for i in range(0, num_sentences, chunk_size):
                chunk = '  '.join(sentences[i:i+chunk_size])
                chunks.append(chunk)
    
    return chunks

Using Map Reduce Chain Type

In [44]:
question_prompt_template = """
Understand from the context how instructors support student motivation by provifing rationales, 
relevance to the real world or enthusiasm in a classroom. The question is a transcript of a lecture. 
Classify every question as rationale, relevance, enthusiasm or none based on the how the concepts
have been described in the context. 
Return the exact verbatim of the question, its classification and why it was classified as such. 
DO NOT change the words in the question, keep it as is.
{context}
Question: {question}
Relevant text, if any:"""
QUESTION_PROMPT = PromptTemplate(
    template=question_prompt_template, input_variables=["context", "question"]
)

combine_prompt_template = """

Given the question and a classification of the question, print the verbatim of the question, your final classification based on the 
context and the justification for the classification. Print the question even when you classify it as None.


QUESTION: {question}
=========
{summaries}
=========
Answer:"""
COMBINE_PROMPT = PromptTemplate(
    template=combine_prompt_template, input_variables=["summaries", "question"]
)


chunks = chunk_text_by_sentence("lab transcript (1).docx.txt", 5)  # Calling function to chunk the pdf by sentence


qa_chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_reduce", return_map_steps=False, question_prompt=QUESTION_PROMPT, combine_prompt=COMBINE_PROMPT)

qa = RetrievalQA(combine_documents_chain=qa_chain, retriever=retriever)

for chunk in chunks:
    result = qa({"input_documents":db,"query": chunk}, return_only_outputs=True)
    print(result['result'])
    print("-----------------------------------------------")


 QUESTION: ﻿Okay great!  So yeah, so let’s please settle down.  I’m going to get started.  Great.  So just a couple of quick announcements here.
Classification: None
Explanation: This is not a question, it is an instruction to the class to settle down and for the instructor to start the lecture. Therefore, it does not fit into any of the categories.
-----------------------------------------------

Question: You have your quiz due on Friday just as usual. You have your review on Friday as well during the class time as usual. we will not be able to finish all the problems that are posted for this week today. Okay just because Monday’s class was cancelled so that does impact how many problems we can do today. So, what I will do is leave the rest of the problems and we continue with them next week because lecture 15 there are not so many problems that we will have.
Classification: Rationale
Explanation: The instructor is providing a rationale for why the students will not be able to finish



Question: “I wish I could get you guys to understand that speed is about more than cars” - Donna
Classification: Enthusiasm
Why: This question is classified as enthusiasm because it is an example of the instructor expressing enthusiasm for the topic and trying to get the students to understand the importance of the topic.
-----------------------------------------------

Question: Here, look at the electronegativity chart.  Always, anything with nitrogen, oxygen, fluorine, you very quickly would be able to determine that it’s going to be electronegative since those are extremely there like the higher electronegative species okay.  In the final one, there is only one polar bond, it’s carbon and fluorine.  So that single bond is going to be the most electro, the most polar bond as well.  The difference is about 1.5 and any bond with chlorine really is most likely going to be a polar bond okay.
Classification: Relevance
Why: The instructor is providing an example of how the concept of el


Question: So, I'll do the formula for formal charge and then we can we can see from there.
Classification: Rationale
Explanation: The instructor is providing a rationale for why they are doing the formula for formal charge. They are explaining the purpose of the formula and how it will help them to see the results.
-----------------------------------------------
 Carbon will always have 4 valence electrons okay.
Classification: Rationale
Explanation: The instructor is providing a rationale for why carbon has 4 valence electrons.
-----------------------------------------------

Question: So, in this particular case, if I was looking at this particular model, this particular ion, this carbon cation, the overall charge would be plus one in this case.
Classification: Rationale
Reason: The instructor is providing a rationale for why the overall charge of the carbon cation is plus one.
-----------------------------------------------

Question: Okay so, if you’re giving something a formal ch

 Question: How would you know? 
Classification: None
Explanation: This question does not relate to the concepts of providing rationales, relevance to the real world or enthusiasm in a classroom.
-----------------------------------------------

Question: So that will give you an indication that the phosphorus would need to be bonded to those, each of those oxygens.
Classification: Rationale
Why: The instructor is providing a rationale for why the phosphorus needs to be bonded to the oxygens.
-----------------------------------------------

Question: You really are looking at group 14, 15 and 16.
Classification: Enthusiasm
Explanation: The instructor is providing enthusiasm by encouraging the students to practice and emphasizing the importance of the lesson.
-----------------------------------------------


Question: Just keep in mind that three negative means you have to add three extra electrons when you’re counting the valence electrons, that’s what the charge is there for.
Classifica

In [43]:
chunks = chunk_text_by_sentence("lab transcript (1).docx.txt", 5)  # Calling function to chunk the pdf by sentence

for chunk in chunks:
    print(chunk)
    print("-----------------------------------------------")


﻿Okay great!  So yeah, so let’s please settle down.  I’m going to get started.  Great.  So just a couple of quick announcements here.
-----------------------------------------------
You have your quiz due on Friday just as usual.  You have your review on Friday as well during the class time as usual.  we will not be able to finish all the problems that are posted for this week today.  Okay just because Monday’s class was cancelled so that does impact how many problems we can do today.  So, what I will do is leave the rest of the problems and we continue with them next week because lecture 15 there are not so many problems that we will have.
-----------------------------------------------
Okay?  So, I’ll post all the answers together, I’ll actually post this even now just so you have them I’ll post the answers even if we don’t complete them this week.  And then next week, we go over everything we didn’t finish and do the lecture 15 problems.  There is no review next week.  So just to ma